In [336]:
#imports
from qiskit import*
from qiskit.tools.visualization import plot_bloch_multivector
from qiskit.tools.visualization import plot_histogram
import numpy as np
from numpy import linalg as LA
import random
from qiskit.extensions.simulator import snapshot
from matplotlib import pyplot as plt
from sortedcontainers import SortedDict
from qiskit.extensions.simulator import snapshot
from qiskit.providers.aer import noise
%matplotlib inline

In [337]:
# Choose a real device to simulate
IBMQ.load_account()
provider = IBMQ.get_provider(group='open')
device = provider.get_backend('ibmq_16_melbourne')
properties = device.properties()
coupling_map = device.configuration().coupling_map

# Generate an Aer noise model for device
noise_model = noise.device.basic_device_noise_model(properties)
basis_gates = noise_model.basis_gates


C:\an\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:181: UserWarning: Credentials are already in use. The existing account in the session will be replaced.
  warnings.warn('Credentials are already in use. The existing '


In [339]:
def probabilty_dist(state):
    probability_list = []
    for i in range(len(state)): 
        probability_list.append(np.absolute(state[i])**2)
    return probability_list

In [340]:
def fidelity(initial_prob_list,final_prob_list):
    fidelity = 0
    for i in range(len(initial_prob_list)):
        fidelity += np.sqrt(initial_prob_list[i]*final_prob_list[i])
    return fidelity**2

In [341]:
def normalize(vector):
    return vector / LA.norm(vector)

def random_initialize(circuit,no_q):
    initial_prob_list = []
    state_list = []
    for i in range(no_q):
        state = np.array([complex(random.random(),random.random()),complex(random.random(),random.random())])
        state = normalize(state)
        state_list.append(state)
        circuit.initialize(state.tolist(),i)
    initial_state = state_list[0]
    for i in range(1,no_q):
        initial_state =  np.kron(state_list[i],initial_state)
    initial_prob_list = probabilty_dist(initial_state)
    return initial_prob_list


In [342]:
def scramble(circuit,no_g):
    l = []
    #scramble
    for i in range(no_g):
        n = list(range(no_q))
        a = random.choice(n)
        n.remove(a)
        b = random.choice(n)
        l.append([a,b])
        circuit.cx(a,b)
    circuit.barrier()
    #invert
    for i in range(1,no_g + 1):
        circuit.cx(l[-i][0],l[-i][1])
    for i in range(no_q):
        circuit.measure(i,i)
    circuit.barrier()
    

In [374]:
def benchmark(no_q,no_g,circuit):
    initial_prob_list = random_initialize(circuit,no_q)
    scramble(circuit,no_g)
    return initial_prob_list
    


In [520]:
no_q = 10 #no of qubits
no_g = 10 #no of gates
shots = 100
circuit = QuantumCircuit(no_q,no_q)
initial_prob_list = benchmark(no_q,no_g,circuit)

In [521]:

simulator = Aer.get_backend('qasm_simulator')
result = execute(circuit,simulator,shots = shots,coupling_map=coupling_map,
                 noise_model=noise_model,
                 basis_gates=basis_gates).result()

In [522]:
b = result.get_counts()#fix list size error 
keys = []
b = dict((int(key), value) for (key, value) in b.items())
for i in range(len(initial_prob_list)):
    keys.append(int(bin(i)[2:]))
for k in keys:
    if (k not in b.keys()):
        b.update({k:0})
        
b = SortedDict(b)
final_prob_list = np.array(b.values())/shots


In [523]:
print(fidelity(initial_prob_list,final_prob_list))

0.04097266747546663


32


In [454]:


print(keys)

[0, 1, 10, 11, 100, 101, 110, 111, 1000, 1001, 1010, 1011, 1100, 1101, 1110, 1111, 10000, 10001, 10010, 10011, 10100, 10101, 10110, 10111, 11000, 11001, 11010, 11011, 11100, 11101, 11110, 11111]


In [480]:
a = [1,2,3]
if( 5 not in a):
    print('yes')

yes


In [505]:
a = dict((int(key), value) for (key, value) in b.items())
print(a.keys())

dict_keys([10001, 11101, 111, 11, 11001, 1011, 10010, 11100, 0, 10, 11000, 11011, 1100, 11111, 10111, 10100, 110, 100, 11110, 10000, 1, 10011, 11010, 1101, 1001, 1111, 1010, 10101, 101, 1000, 10110])
